In [ ]:
"""
%pip install matplotlib==3.10.1
%pip install pennylane==0.42.3
%pip install numpy==1.26.4
%pip install pandas==2.2.2
%pip install basis-set-exchange
%pip install qiskit-aer==0.17.1
%pip install pennylane-qiskit==0.40.1
%pip install qiskit-aer-gpu==0.15.1
%pip install pyswarms==1.3.0
%pip install qiskit-ibm-runtime==0.35.0
"""

'\n%pip install matplotlib==3.10.1\n%pip install pennylane==0.42.3\n%pip install numpy==1.26.4\n%pip install pandas==2.2.2\n%pip install basis-set-exchange\n%pip install qiskit-aer==0.17.1\n%pip install pennylane-qiskit==0.40.1\n%pip install qiskit-aer-gpu==0.15.1\n%pip install pyswarms==1.3.0\n%pip install qiskit-ibm-runtime==0.35.0\n'

In [ ]:
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger('qiskit').setLevel(logging.WARNING)

# 1º: Create the Molecule and Hamiltonian

In the code cells below, we create the molecular geometry of H2 with the radial distance of 1,401 a.u. with the optimal internuclear distance of Szabo and Ostlund. We will use the STO-3G minimal basis set by importing the parameters of the basis set exchange functions. Pennylane allows us to construct the Hamiltonian from the second quantization with the Jordan-Wigner transformation.

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeMarrakesh
from pyswarms.backend.topology import Star
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
from functools import partial
from collections import deque
import pyswarms.backend as P
import pennylane as qml
import pyswarms as ps
import pandas as pd
import numpy as np
import time
import json

symbols = ["H", "H"]
coordinates = np.array([[0.0, 0.0, 0], [0.0, 0.0, 1.401]])
molecule = qml.qchem.Molecule(symbols, coordinates, basis_name='STO-3G', load_data=True, unit='bohr')
H, qubits = qml.qchem.molecular_hamiltonian(molecule)

print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

2025-08-25 02:11:39,129 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


Number of qubits =  4
The Hamiltonian is  -0.09883485860187924 * I([0, 1, 2, 3]) + 0.17120123803197834 * Z(0) + 0.17120123803197845 * Z(1) + 0.16862327620358059 * (Z(0) @ Z(1)) + -0.22279639115527738 * Z(2) + 0.12054612718324412 * (Z(0) @ Z(2)) + 0.16586801098505832 * (Z(1) @ Z(2)) + 0.045321883801814206 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.045321883801814206 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.045321883801814206 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.045321883801814206 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.22279639115527738 * Z(3) + 0.16586801098505832 * (Z(0) @ Z(3)) + 0.12054612718324412 * (Z(1) @ Z(3)) + 0.17434948668373768 * (Z(2) @ Z(3))


# 2º: Calculate the Number of Shots Necessary for the Experiment

In the code cells below, we calculate the number of shots required for our noise-free experiment using the PSO optimizer. This calculation is based on sampling formulas and a specified precision, denoted by 𝜖. After determining the required number of shots, we round it to 4,000,000. Increasing the precision will result in longer computation times and may also require more memory.

In [ ]:
e = 1e-3
cj = H.terms()[0]

sum_squarred = sum([abs(float(c)) for c in cj])**2

shots_necessary = round(1/(e**2)*(sum_squarred))
shots_necessary

3935933

In [ ]:
shots_necessary=4000000

# 3º: Configure the Simulator

In the cells below, we configure a noisy Qiskit Aer simulator using the PennyLane-Qiskit plugin. Our goal is to simulate the UCCSD ansatz with PennyLane. The quantum circuit is designed to compute the expected value of the Hamiltonian by collecting a large number of samples obtained from the shots. For these simulations, we will leverage the GPU to accelerate the computations.

In [ ]:
fake_provider = FakeMarrakesh()
aer_simulator = AerSimulator.from_backend(backend=fake_provider,method="density_matrix",device='GPU')

In [ ]:
dev = qml.device("qiskit.aer", backend=aer_simulator, wires=qubits, shots=shots_necessary)

In [ ]:
electrons = 2
hf_state = qml.qchem.hf_state(electrons, qubits)
singles, doubles = qml.qchem.excitations(electrons, qubits)
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

In [ ]:
@qml.qnode(dev)
def circuit(params):
    wires = range(qubits)
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)

In [ ]:
def cost_fn(swarm_positions):
    swarm_costs = [circuit(position) for position in swarm_positions]
    return swarm_costs

# 4°: Configure the optimizer and run the circuit

The PSO configuration is done manually with the help of the PySwarms library. In this experiment, we used the star topology configuration to search for GBEST.

In [ ]:
hist = {
    "particles_velocity_hist": [],
    "particles_position_hist": [],
    "particles_cost_hist": [],
    "particles_pbest_cost": [],
    "particles_pbest_pos":[],
    "swarm_global_best": [],
    "swarm_best_position": [],
}

In [ ]:
def save_history(
    particles_velocity: list,
    particles_position: list,
    particles_cost: list,
    particles_pbest_cost: list,
    particles_pbest_pos: list,
    swarm_global_best: float,
    swarm_best_position: list
) -> None:

    hist["particles_velocity_hist"].append(particles_velocity)
    hist["particles_position_hist"].append(particles_position)
    hist["particles_cost_hist"].append(particles_cost)
    hist["particles_pbest_cost"].append(particles_pbest_cost)
    hist["particles_pbest_pos"].append(particles_pbest_pos)
    hist["swarm_global_best"].append(swarm_global_best)
    hist["swarm_best_position"].append(swarm_best_position)

In [ ]:
topology = Star()
options = {'c1':1.43, 'c2': 1.43, 'w':0.8}
swarm = P.create_swarm(n_particles=15, dimensions=3, options=options)

In [ ]:
ftol = 1e-6
ftol_iter = 10
ftol_hist = deque(maxlen=ftol_iter)

In [ ]:
n_particles, n_dims = swarm.position.shape

# pbest e gbest inicial
swarm.pbest_cost = np.full(n_particles, np.inf, dtype=float)
swarm.pbest_pos  = swarm.position.copy()
swarm.best_cost  = np.inf
swarm.best_pos   = swarm.position[0].copy()

In [ ]:
for i in range(100):
    # 1) avalia custo nas posições atuais
    swarm.current_cost = np.asarray(cost_fn(swarm.position), dtype=float)

    # 2) atualiza pbest
    swarm.pbest_pos, swarm.pbest_cost = P.compute_pbest(swarm)

    # 3) guarda melhor anterior e atualiza gbest
    best_cost_prev = getattr(swarm, "best_cost", np.inf)
    swarm.best_pos, swarm.best_cost = topology.compute_gbest(swarm)

    # 4) critério de parada (mesma lógica do PySwarms)
    if np.isfinite(best_cost_prev):
        relative_measure = ftol * (1.0 + abs(best_cost_prev))
        delta = abs(swarm.best_cost - best_cost_prev) < relative_measure  # True => não melhorou o suficiente
    else:
        delta = False  # primeira iteração (ou anterior = inf), não paramos

    ftol_hist.append(delta)
    if len(ftol_hist) == ftol_iter and all(ftol_hist):
        # opcional: salvar estado final antes de sair
        save_history(
            particles_velocity=swarm.velocity,
            particles_position=swarm.position,
            particles_cost=swarm.current_cost,
            particles_pbest_cost=swarm.pbest_cost,
            particles_pbest_pos=swarm.pbest_pos,
            swarm_global_best=swarm.best_cost,
            swarm_best_position=swarm.best_pos
        )
        print(f"Early stop at iteration {i+1}: best_cost = {swarm.best_cost:.8f}")
        break

    # 5) atualiza velocidade e posição
    swarm.velocity = topology.compute_velocity(swarm)
    swarm.position = topology.compute_position(swarm)

    save_history(
        particles_velocity=swarm.velocity,
        particles_position=swarm.position,
        particles_cost=swarm.current_cost,
        particles_pbest_cost=swarm.pbest_cost,
        particles_pbest_pos=swarm.pbest_pos,
        swarm_global_best=swarm.best_cost,
        swarm_best_position=swarm.best_pos
    )

    print(f"Iteration: {i+1} | best_cost: {swarm.best_cost:.8f}")

Iteration: 1 | best_cost: -0.79266268
Iteration: 2 | best_cost: -0.79266268
Iteration: 3 | best_cost: -0.80623327
Iteration: 4 | best_cost: -0.85363719
Iteration: 5 | best_cost: -0.85363719
Iteration: 6 | best_cost: -0.85881937
Iteration: 7 | best_cost: -0.86124807
Iteration: 8 | best_cost: -0.86160731
Iteration: 9 | best_cost: -0.86160731
Iteration: 10 | best_cost: -0.86302762
Iteration: 11 | best_cost: -0.86302762
Iteration: 12 | best_cost: -0.86325148
Iteration: 13 | best_cost: -0.86325148
Iteration: 14 | best_cost: -0.86325148
Iteration: 15 | best_cost: -0.86345664
Iteration: 16 | best_cost: -0.86345664
Iteration: 17 | best_cost: -0.86345664
Iteration: 18 | best_cost: -0.86345664
Iteration: 19 | best_cost: -0.86370940
Iteration: 20 | best_cost: -0.86370940
Iteration: 21 | best_cost: -0.86370940
Iteration: 22 | best_cost: -0.86370940
Iteration: 23 | best_cost: -0.86370940
Iteration: 24 | best_cost: -0.86370940
Iteration: 25 | best_cost: -0.86370940
Iteration: 26 | best_cost: -0.8638

In [ ]:
best_cost = swarm.best_cost
best_pos = swarm.best_pos

print(f"Best Energy Found By Swarm: {best_cost:.8f} Ha at Position: {best_pos}")

Best Energy Found By Swarm: -0.86380376 Ha at Position: [-0.00918113 -0.01954317  0.13234449]


# 5º: Save the Data for Later Analysis

In [ ]:
history_data = pd.DataFrame({
    "particles_velocity_hist": hist['particles_velocity_hist'],
    "particles_position_hist": hist['particles_position_hist'],
    "particles_cost_hist": hist['particles_cost_hist'],
    "particles_pbest_cost": hist['particles_pbest_cost'],
    "particles_pbest_pos": hist['particles_pbest_pos'],
    "swarm_global_best": hist['swarm_global_best'],
    "swarm_best_position": hist['swarm_best_position']

})


history_data["particles_velocity_hist_json"] = history_data["particles_velocity_hist"].apply(lambda arr: json.dumps(arr.tolist()))
history_data["particles_position_hist_json"] = history_data["particles_position_hist"].apply(lambda arr: json.dumps(arr.tolist()))
history_data["particles_cost_hist_json"] = history_data["particles_cost_hist"].apply(lambda arr: json.dumps(arr.tolist()))
history_data["particles_pbest_cost_json"] = history_data["particles_pbest_cost"].apply(lambda arr: json.dumps(arr.tolist()))
history_data["particles_pbest_pos_json"] = history_data["particles_pbest_pos"].apply(lambda arr: json.dumps(arr.tolist()))
history_data["swarm_global_best_json"] = history_data["swarm_global_best"].apply(lambda arr: json.dumps(arr))
history_data["swarm_best_position_json"] = history_data["swarm_best_position"].apply(lambda arr: json.dumps(arr.tolist()))

In [ ]:
columns = [
    "particles_velocity_hist_json",
    "particles_position_hist_json",
    "particles_cost_hist_json",
    "particles_pbest_cost_json",
    "particles_pbest_pos_json",
    "swarm_global_best_json",
    "swarm_best_position_json"
]

history_data.to_csv("PSO_GBEST_NOISY.csv", columns=columns, index=False)